
        select bank_id, transaction_type, swtc+swtv total, wight
        from(
            select bank_id, transaction_type, sum(wight) wight , sum(transaction_value) swtv, sum(transaction_cost) swtc
            from(
                select bank_id, transaction_type, sum(new_count) wight, transaction_cost, transaction_value
                from(
                    select *
                    from(
                        select bank_id, transaction_type,  count(count) new_count, transaction_cost, transaction_value
                        from data_dataset
                        where count = 0
                        group by bank_id, transaction_type, transaction_cost, transaction_value) as b0
 
                    union all
                    select *
                    from(
                        select bank_id, transaction_type,  count new_count, transaction_cost, transaction_value
                        from data_dataset
                        where count != 0) as b1) as b2
                group by bank_id, transaction_type, transaction_cost, transaction_value) as b3
            group by bank_id, transaction_type) as b4
            where bank_id = {{number_of_bankid}}
            order by total desc

In [11]:
import pandas as pd

In [12]:
data = pd.read_csv('../../github_source/tranasaction_dataset.csv', parse_dates=['time_internal'])

In [13]:
tmp_data = data[data['count'] == 0]
dg1 = tmp_data.groupby(by=['bank_id', 'transaction_type', 'transaction_cost', 'transaction_value']).agg({'count': 'count'}).reset_index()

In [14]:
tmp_data2 = data[data['count'] != 0]
dg2 = tmp_data2[['count', 'bank_id', 'transaction_type', 'transaction_cost', 'transaction_value']]

In [15]:
dg = pd.concat([dg1, dg2])

In [16]:
dg = dg.groupby(by=['bank_id', 'transaction_type', 'transaction_cost', 'transaction_value']).agg({'count': 'sum'}).reset_index()

In [17]:
dg = dg.groupby(by=['bank_id', 'transaction_type']).agg({
    'count' : 'sum',
    'transaction_cost' : 'sum',
    'transaction_value' : 'sum'
}).reset_index()

In [18]:
dg['total_value'] = dg['transaction_cost'] + dg['transaction_value']
dg = dg.drop(['transaction_cost', 'transaction_value'], axis='columns')

In [19]:
your_bank_id = 13335
dg[dg['bank_id'] == your_bank_id].sort_values('total_value', ascending=False)

,bank_id,transaction_type,count,total_value
4106,13335,28691,696619,50649614752
3974,13335,850,10496,22017281934
4095,13335,28412,200070,8150066745
4007,13335,27176,16773,3296677561
4092,13335,28392,32511,1046757532
...,...,...,...,...
4080,13335,27981,3,492
4063,13335,27848,5,445
3994,13335,27027,5,385
4060,13335,27811,2,370
